In [ ]:
!pip install -U tensorflow_hub
!pip install -U tensorflow_datasets

In [2]:
import time
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow.keras import layers

In [3]:
(train_examples, validation_examples), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)
def format_image(image, label):
  # `hub` image modules exepct their data normalized to the [0,1] range.
  image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/255.0
  return  image, label

num_examples = info.splits['train'].num_examples

BATCH_SIZE = 32
IMAGE_RES = 224

train_batches      = train_examples.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.cache().map(format_image).batch(BATCH_SIZE).prefetch(1)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteGLECMR/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [5]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMAGE_RES, IMAGE_RES,3))
feature_extractor.trainable = False

model = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(2)
])
model.compile(
  optimizer='adam', 
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

EPOCHS = 3
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 743s 1s/step - loss: 0.0541 - accuracy: 0.9813 - val_loss: 0.0312 - val_accuracy: 0.9895
Epoch 2/3
582/582 [==============================] - 740s 1s/step - loss: 0.0300 - accuracy: 0.9903 - val_loss: 0.0310 - val_accuracy: 0.9888
Epoch 3/3
582/582 [==============================] - 738s 1s/step - loss: 0.0254 - accuracy: 0.9915 - val_loss: 0.0308 - val_accuracy: 0.9903


## Now that we've trained the model,  we can save it as an HDF5 file, which is the format used by Keras. Our HDF5 file will have the extension '.h5', and it's name will correpond to the current time stamp.

In [6]:
t = time.time()

export_path_keras = "./{}.h5".format(int(t))
model.save(export_path_keras)

#Load the Keras `.h5` Model

We will now load the model we just saved into a new model called `reloaded`. We will need to provide the file path and the `custom_objects` parameter. This parameter tells keras how to load the `hub.KerasLayer` from the `feature_extractor` we used for transfer learning.

In [7]:
reloaded = tf.keras.models.load_model(
  export_path_keras, 
  custom_objects={'KerasLayer': hub.KerasLayer})

reloaded.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


# Keep Training

Besides making predictions, we can also take our `reloaded` model and keep training it. To do this, you can just train the `reloaded` as usual, using the `.fit` method.

In [8]:
EPOCHS = 3
history = reloaded.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 749s 1s/step - loss: 0.0210 - accuracy: 0.9932 - val_loss: 0.0309 - val_accuracy: 0.9905
Epoch 2/3
582/582 [==============================] - 749s 1s/step - loss: 0.0198 - accuracy: 0.9929 - val_loss: 0.0311 - val_accuracy: 0.9903
Epoch 3/3
582/582 [==============================] - 749s 1s/step - loss: 0.0173 - accuracy: 0.9940 - val_loss: 0.0331 - val_accuracy: 0.9910


# Export as SavedModel


In [9]:
t = time.time()
export_path_sm = "./{}".format(int(t))
tf.saved_model.save(model, export_path_sm)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ./1593002202/assets


INFO:tensorflow:Assets written to: ./1593002202/assets


# Load SavedModel

In [10]:
reloaded_sm = tf.saved_model.load(export_path_sm)

# Loading the SavedModel as a Keras Model

The object returned by `tf.saved_model.load` is not a Keras object (i.e. doesn't have `.fit`, `.predict`, `.summary`, etc. methods). Therefore, you can't simply take your `reloaded_sm` model and keep training it by running `.fit`. To be able to get back a full keras model from the Tensorflow SavedModel format we must use the `tf.keras.models.load_model` function. This function will work the same as before, except now we pass the path to the folder containing our SavedModel.

In [11]:
t = time.time()

export_path_sm = "./{}".format(int(t))
tf.saved_model.save(model, export_path_sm)

INFO:tensorflow:Assets written to: ./1593002211/assets


INFO:tensorflow:Assets written to: ./1593002211/assets


In [12]:
reload_sm_keras = tf.keras.models.load_model(
  export_path_sm,
  custom_objects={'KerasLayer': hub.KerasLayer})

reload_sm_keras.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 1280)              2257984   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,226,434
Non-trainable params: 34,112
_________________________________________________________________


In [14]:
image_batch, label_batch = next(iter(train_batches.take(1)))
image_batch = image_batch.numpy()
result_batch = model.predict(image_batch)
reload_sm_keras_result_batch = reload_sm_keras.predict(image_batch)

In [15]:
(abs(result_batch - reload_sm_keras_result_batch)).max()

0.0

#Download your model

In [16]:
!zip -r model.zip {export_path_sm}

  adding: 1593002211/ (stored 0%)
  adding: 1593002211/saved_model.pb (deflated 92%)
  adding: 1593002211/variables/ (stored 0%)
  adding: 1593002211/variables/variables.index (deflated 78%)
  adding: 1593002211/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: 1593002211/assets/ (stored 0%)


In [17]:
try:
  from google.colab import files
  files.download('./model.zip')
except ImportError:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>